# Converters for Quadratic Programs
The latest version of this notebook is available on https://github.com/Qiskit/qiskit-iqx-tutorials.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#InequalityToEqualityConverter" data-toc-modified-id="InequalityToEqualityConverter-2"><code>InequalityToEqualityConverter</code></a></span><ul class="toc-item"><li><span><a href="#A-usage-example-of-InequalityToEqualityConverter" data-toc-modified-id="A-usage-example-of-InequalityToEqualityConverter-2.1">A usage example of <code>InequalityToEqualityConverter</code></a></span></li></ul></li><li><span><a href="#IntegerToBinaryConverter" data-toc-modified-id="IntegerToBinaryConverter-3"><code>IntegerToBinaryConverter</code></a></span><ul class="toc-item"><li><span><a href="#A-usage-example-of-IntegerToBinaryConverter" data-toc-modified-id="A-usage-example-of-IntegerToBinaryConverter-3.1">A usage example of <code>IntegerToBinaryConverter</code></a></span></li></ul></li><li><span><a href="#PenalizeLinearEqualityConstraints" data-toc-modified-id="PenalizeLinearEqualityConstraints-4"><code>PenalizeLinearEqualityConstraints</code></a></span><ul class="toc-item"><li><span><a href="#A-usage-example-of-PenalizeLinearEqualityConstraints" data-toc-modified-id="A-usage-example-of-PenalizeLinearEqualityConstraints-4.1">A usage example of <code>PenalizeLinearEqualityConstraints</code></a></span></li></ul></li><li><span><a href="#Checking-the-answer-of-the-converted-QuadraticProgram" data-toc-modified-id="Checking-the-answer-of-the-converted-QuadraticProgram-5">Checking the answer of the converted <code>QuadraticProgram</code></a></span></li><li><span><a href="#(Optional)-OptimizationProblemToQubo-converter" data-toc-modified-id="(Optional)-OptimizationProblemToQubo-converter-6">(Optional) <code>OptimizationProblemToQubo</code> converter</a></span><ul class="toc-item"><li><span><a href="#A-usage-example-of-OptimizationProblemToQubo" data-toc-modified-id="A-usage-example-of-OptimizationProblemToQubo-6.1">A usage example of <code>OptimizationProblemToQubo</code></a></span></li></ul></li></ul></div>

## Introduction
In order to solve an optimization problem with quantum computer, we need to map the optimization problem to a corresponding Ising Hamiltonian. To do so, firstly, it's necessary to convert the optimization problem into one with a specific form, more precisely an Unconstrained Binary Optimization form. Otherwise we can not create the corresponding Ising Hamiltonian from the optimization problem. However, the converting process is complicated and it requires specialized knowledges.

For that purpose, Qiskit Aqua has three converters. With these converters, optimization problems can be easily and automatically converted into one with a specific form to solve with quantum computers. The following is a list of three converters. The details of each converter will be discussed later with examples.
- `InequalityToEqualityConverter`: converts inequality constraints into equality constraints with additional slack variables.
- `IntegerToBinaryConverter`: converts integer variables into binary variables and corresponding coefficients. 
- `PenalizeLinearEqualityConstraints`: convert equality constraints into additional terms of the object function.

Also, we recommend to apply each converter in the following order.
1. `InequalityToEqualityConverter`
2. `IntegerToBinaryConverter`
3. `PenalizeLinearEqualityConstraints`

## `InequalityToEqualityConverter`
`InequalityToEqualityConverter` converts inequality constraints into equality constraints with additional slack variables to remove inequality constraints from `QuadraticProgram`. The upper bounds and the lower bounds of slack variables will be calculated from the difference between the left sides and the right sides of constraints. Signs of slack variables depend on symbols in constraints such as $\leq$ and $\geq$.

### A usage example of `InequalityToEqualityConverter`
The following is a toy example of a maximization problem with an inequality constraint. Variable $x$ and $y$ are binary variables. variable $z$ is an integer variable.

\begin{aligned}
   & \text{maximize}
       & 2x + y + z\\
   & \text{subject to}
       & x+y+z \leq 5\\
       & & x, y \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5,6,7\} \\
\end{aligned}

With `QuadraticProgram`, an optimization model of the problem is written as follows.

In [1]:
from qiskit.optimization import OptimizationProblem
from cplex import SparsePair

op = OptimizationProblem()
op.variables.add(names=['x', 'y', 'z'], types='BBI', lb=[0, 0, 0], ub=[1, 1, 7])
op.objective.set_linear([('x', 2),('y', 1),('z', 1)])
op.linear_constraints.add(
    lin_expr=[SparsePair(ind=['x', 'y', 'z'], val=[1, 1, 1])],
    senses=['L'],
    rhs=[5],
    names=['xyz']
)
op.objective.set_sense(-1)
print(op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z
Subject To
 xyz: x + y + z <= 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 7
Binaries
 x  y 
Generals
 z 
End



Call `encode` method of `InequalityToEqualityConverter` to convert.

In [2]:
from qiskit.optimization.converters import InequalityToEqualityConverter

ineq2eq_conv = InequalityToEqualityConverter()
new_op = ineq2eq_conv.encode(op)
print(new_op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z
Subject To
 xyz: x + y + z + xyz@int_slack  = 5
Bounds
 0 <= x <= 1
 0 <= y <= Inf
 0 <= z <= 7
 0 <= xyz@int_slack <= 5
Binaries
 x  y 
Generals
 z  xyz@int_slack 
End



After converting, the inequality constraint is replaced with an equality constraint with an additional slack variable as the above. 

## `IntegerToBinaryConverter`

`IntegerToBinaryConverter` converts integer variables into binary variables and coefficients to remove integer variables from `QuadraticProgram`. For converting, bounded-coefficient encoding proposed in arxiv:1706.01945 (Eq. (5)) is used. For more detail of the encoding method, please see the paper.

### A usage example of `IntegerToBinaryConverter`
The following is a toy example of a maximization problem with an integer variable. The following example is also the output of the above `InequalityToEqualityConverter` example. Variable $x$, $y$ and $xyz\text{@}int_slack$ are binary variables. variable $z$ is an integer variable.

\begin{aligned}
   & \text{maximize}
       & 2x + y + z\\
   & \text{subject to}
       & x+y+z +xyz\text{@}int\_slack= 5\\
       & & x, y  \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5,6,7\} \\
       & & xyz\text{@}int\_slack \in \{0,1,2,3,4,5\} \\
\end{aligned}

With `QuadraticProgram`, an optimization model of the problem is written as follows.

In [3]:
op = OptimizationProblem()
op.variables.add(names=['x', 'y', 'z','xyz@int_slack'], types='BBII', lb=[0, 0, 0, 0], ub=[1, 1, 7, 5])
op.objective.set_linear([('x', 2),('y', 1),('z', 1)])
op.linear_constraints.add(
    lin_expr=[SparsePair(ind=['x', 'y', 'z', 'xyz@int_slack'], val=[1, 1, 1, 1])],
    senses=['E'],
    rhs=[5],
    names=['xyz']
)
op.objective.set_sense(-1)
print(op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z
Subject To
 xyz: x + y + z + xyz@int_slack  = 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 7
 0 <= xyz@int_slack <= 5
Binaries
 x  y 
Generals
 z  xyz@int_slack 
End



Call `encode` method of `IntegerToBinaryConverter` to convert.

In [4]:
from qiskit.optimization.converters import IntegerToBinaryConverter

int2bin_conv = IntegerToBinaryConverter()
new_op = int2bin_conv.encode(op)
print(new_op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z@0 + 2 z@1 + 4 z@2
Subject To
 xyz: x + y + z@0 + 2 z@1 + 4 z@2 + xyz@int_slack@0 + 2 xyz@int_slack@1
      + 2 xyz@int_slack@2  = 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z@0 <= 1
 0 <= z@1 <= Inf
 0 <= z@2 <= Inf
 0 <= xyz@int_slack@0 <= Inf
 0 <= xyz@int_slack@1 <= Inf
 0 <= xyz@int_slack@2 <= Inf
Binaries
 x  y  z@0  z@1  z@2  xyz@int_slack@0  xyz@int_slack@1  xyz@int_slack@2 
End



After converting, integer variables $z$ is replaced with three binary variables $z\text{@}0$, $z\text{@}1$ and $z\text{@}2$ with coefficients 1, 2 and 4, respectively as the above. The slack variable $xyz\text{@}int\_slack$ that was introduced by `InequalityToEqualityConverter` is also replaced with $xyz\text{@}int\_slack\text{@}0$, $xyz\text{@}int\_slack\text{@}1$ and $xyz\text{@}int\_slack\text{@}2$ with coefficients 1, 2 and 2 respectively. 

Note: Essentially coefficients mean that the sum of these binary variables with coefficients can be the sum of a subset of {1, 2, 4}  for the case of variable $z$ which will a number from 0 to 7. This respects the lower bound and the upper bound of original integer variable $z$ correctly. The basic concept is similar for the case of variable $xyz\text{@}int\_slack$.

## `PenalizeLinearEqualityConstraints`

`PenalizeLinearEqualityConstraints` converts equality constraints into additional terms of the object function to make `QuadraticProgram` to unconstrained form. An input to the convert has to be a `QuadraticProgram` with only equality constraints. Those equality constraints, e.g. $\sum_i a_i x_i  = b$ where $a_i$ and $b$ are numbers and $x_i$ is a variable, will be added to the objective function in the form of $M(b - \sum_i a_i x_i)^2$ where $M$ is a large number as penalty factor. By default it's 1e5. A sign of the term will depends on the problem type is maximization or minimization.

### A usage example of `PenalizeLinearEqualityConstraints`
The following is a toy example of a maximization problem with equality constraints. The following example is also the output of the above `IntegerToBinaryConverter` example. The all variables are binary variables now. 

\begin{aligned}
   & \text{maximize}
       & 2x + y + z\text{@}0 + 2 z\text{@}1 + 4 z\text{@}2\\
   & \text{subject to}
       & x + y+ z\text{@}0 + 2 z\text{@}1 + 4 z\text{@}2 + xyz\text{@}int\_slack@0 +2 xyz\text{@}int\_slack@1 + 2 xyz\text{@}int\_slack@2 = 5\\
       & & x, y, z\text{@}0 + z\text{@}1 + z\text{@}2 + xyz\text{@}int\_slack\text{@}0 + xyz\text{@}int\_slack\text{@}1 + xyz\text{@}int\_slack\text{@}2 \in \{0,1\}\\
\end{aligned}

With `QuadraticProgram`, an optimization model of the problem is written as follows.

In [5]:
op = OptimizationProblem()
op.variables.add(names=['x', 'y', 'z@0', 'z@1', 'z@2', 'xyz@int_slack@0', 'xyz@int_slack@1', 'xyz@int_slack@2'], types='B' *8)
op.objective.set_linear([('x', 2),('y', 1),('z@0', 1), ('z@1', 2), ('z@2', 4)])
op.linear_constraints.add(
    lin_expr=[SparsePair(ind=['x', 'y', 'z@0', 'z@1','z@2', 'xyz@int_slack@0', 'xyz@int_slack@1', 'xyz@int_slack@2'], val=[1, 1, 1, 2, 4, 1, 2, 2])],
    senses=['E'],
    rhs=[5],
    names=['xyz']
)
op.objective.set_sense(-1)
print(op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z@0 + 2 z@1 + 4 z@2
Subject To
 xyz: x + y + z@0 + 2 z@1 + 4 z@2 + xyz@int_slack@0 + 2 xyz@int_slack@1
      + 2 xyz@int_slack@2  = 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z@0 <= 1
 0 <= z@1 <= 1
 0 <= z@2 <= 1
 0 <= xyz@int_slack@0 <= 1
 0 <= xyz@int_slack@1 <= 1
 0 <= xyz@int_slack@2 <= 1
Binaries
 x  y  z@0  z@1  z@2  xyz@int_slack@0  xyz@int_slack@1  xyz@int_slack@2 
End



Call encode method of `PenalizeLinearEqualityConstraints` to convert.

In [6]:
from qiskit.optimization.converters import PenalizeLinearEqualityConstraints

penilize_conv = PenalizeLinearEqualityConstraints()
new_op = penilize_conv.encode(op)
print(new_op.write_as_string())

Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 1000002 x + 1000001 y + 1000001 z@0 + 2000002 z@1 + 4000004 z@2
            + 1000000 xyz@int_slack@0 + 2000000 xyz@int_slack@1
            + 2000000 xyz@int_slack@2 + [ - 200000 x ^2 - 400000 x * y
            - 400000 x * z@0 - 800000 x * z@1 - 1600000 x * z@2
            - 400000 x * xyz@int_slack@0 - 800000 x * xyz@int_slack@1
            - 800000 x * xyz@int_slack@2 - 200000 y ^2 - 400000 y * z@0
            - 800000 y * z@1 - 1600000 y * z@2 - 400000 y * xyz@int_slack@0
            - 800000 y * xyz@int_slack@1 - 800000 y * xyz@int_slack@2
            - 200000 z@0 ^2 - 800000 z@0 * z@1 - 1600000 z@0 * z@2
            - 400000 z@0 * xyz@int_slack@0 - 800000 z@0 * xyz@int_slack@1
            - 800000 z@0 * xyz@int_slack@2 - 800000 z@1 ^2 - 3200000 z@1 * z@2
            - 800000 z@1 * xyz@int_slack@0 - 1600000 z@1 * xyz@int_slack@1
            - 1600000 z@1 * xyz@int_slack@2 - 3200000 z@2 ^2
            - 1600000 z@2 * xyz@in

After converting the equality constraint is added to the objective function as additional terms with the default penalty factor 1e5.

## Checking the answer of the converted `QuadraticProgram`

In the end, let's check the answer of the converted `QuadraticProgram` is the same as the original one. 

In [7]:
from qiskit.optimization.algorithms import CplexOptimizer

cplex = CplexOptimizer() 
result = cplex.solve(new_op)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Found incumbent of value -2500000.000000 after 0.01 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve added 56 rows and 28 columns.
Reduced MIP has 56 rows, 36 columns, and 112 nonzeros.
Reduced MIP has 36 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 28 rows and 2 columns.
Reduced MIP has 28 rows, 34 columns, and 82 nonzeros.
Reduced MIP has 34 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.09 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 2.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.04 sec. (

`decode` method of `IntegerToBInaryConverter` is useful to convert back the answer of converted problem to the corresponding original answer.

In [8]:
result = int2bin_conv.decode(result)
print(result)

([1,1,3,0] / 6.0)


The following was the original problem. According to the above answer, the value of objective function is 6. The assignment of variable $x$, $y$ and $z$ are 1, 1, 3, respectively. The assignment satisfy the constraint and the 6 is the maximum number for the objective function as we can easily see. Thus the original problem is converted correctly. 

\begin{aligned}
   & \text{maximize}
       & 2x + y + z\\
   & \text{subject to}
       & x+y+z \leq 5\\
       & & x, y \in \{0,1\}\\
       & & z \in \{0,1,2,3,4,5,6,7\} \\
\end{aligned}

## (Optional) `OptimizationProblemToQubo` converter

We also have `OptimizationProblemToQubo` converter. It contains the functionality of two converters `IntegerToBinaryConverter` and `PenalizeLinearEqualityConstraints` in one for more convenience. It does not contain the functionality of `InequalityToEqualityConverter` though.

### A usage example of `OptimizationProblemToQubo`
The following is a toy example of a maximization problem with an integer variable. The following example is also the output of the above `InequalityToEqualityConverter` example . Variable $x$, $y$ and $xyz\text{@}int\_slack$ are binary variables. variable $z$ is an integer variable.

In [9]:
op = OptimizationProblem()
op.variables.add(names=['x', 'y', 'z','xyz@int_slack'], types='BBII', lb=[0, 0, 0, 0], ub=[1, 1, 7, 5])
op.objective.set_linear([('x', 2),('y', 1),('z', 1)])
op.linear_constraints.add(
    lin_expr=[SparsePair(ind=['x', 'y', 'z', 'xyz@int_slack'], val=[1, 1, 1, 1])],
    senses=['E'],
    rhs=[5],
    names=['xyz']
)
op.objective.set_sense(-1)
print(op.write_as_string())

\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 2 x + y + z
Subject To
 xyz: x + y + z + xyz@int_slack  = 5
Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 7
 0 <= xyz@int_slack <= 5
Binaries
 x  y 
Generals
 z  xyz@int_slack 
End



Call `encode` method of `OptimizationProblemToQubo` to convert in a similar manner as other converters.

In [10]:
from qiskit.optimization.converters import OptimizationProblemToQubo

qubo_conv = OptimizationProblemToQubo()
new_op = qubo_conv.encode(op)
print(new_op.write_as_string())

Default row names c1, c2 ... being created.


\ENCODING=ISO-8859-1
\Problem name: 

Maximize
 Objective: 1000002 x + 1000001 y + 1000001 z@0 + 2000002 z@1 + 4000004 z@2
            + 1000000 xyz@int_slack@0 + 2000000 xyz@int_slack@1
            + 2000000 xyz@int_slack@2 + [ - 200000 x ^2 - 400000 x * y
            - 400000 x * z@0 - 800000 x * z@1 - 1600000 x * z@2
            - 400000 x * xyz@int_slack@0 - 800000 x * xyz@int_slack@1
            - 800000 x * xyz@int_slack@2 - 200000 y ^2 - 400000 y * z@0
            - 800000 y * z@1 - 1600000 y * z@2 - 400000 y * xyz@int_slack@0
            - 800000 y * xyz@int_slack@1 - 800000 y * xyz@int_slack@2
            - 200000 z@0 ^2 - 800000 z@0 * z@1 - 1600000 z@0 * z@2
            - 400000 z@0 * xyz@int_slack@0 - 800000 z@0 * xyz@int_slack@1
            - 800000 z@0 * xyz@int_slack@2 - 800000 z@1 ^2 - 3200000 z@1 * z@2
            - 800000 z@1 * xyz@int_slack@0 - 1600000 z@1 * xyz@int_slack@1
            - 1600000 z@1 * xyz@int_slack@2 - 3200000 z@2 ^2
            - 1600000 z@2 * xyz@in

`OptimizationProblemToQubo` converter also has `decode` method to convert back the answer of the converted problem to the corresponding original answer.

In [11]:
from qiskit.optimization.algorithms import CplexOptimizer

cplex = CplexOptimizer() 
result = cplex.solve(new_op)
original_result = qubo_conv.decode(result)
print(original_result)

Version identifier: 12.10.0.0 | 2019-11-27 | 843d4de
CPXPARAM_Read_DataCheck                          1
Found incumbent of value -2500000.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve added 56 rows and 28 columns.
Reduced MIP has 56 rows, 36 columns, and 112 nonzeros.
Reduced MIP has 36 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Detecting symmetries...
MIP Presolve eliminated 28 rows and 2 columns.
Reduced MIP has 28 rows, 34 columns, and 82 nonzeros.
Reduced MIP has 34 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (0.09 ticks)
Classifier predicts products in MIQP should be linearized.
Probing time = 0.00 sec. (0.01 ticks)
Clique table members: 2.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 4 threads.
Root relaxation solution time = 0.00 sec. (